## Step 0: Import Packages, Mount Drive, and Change File Path

Make sure to change the filepath under the `data` object to your own drive structure

In [0]:
# Install Packages

# Import Packages
import os
from pathlib import Path
import glob
import shutil
import pandas as pd
import re
from progressbar import ProgressBar


In [57]:
# Mount User's Drive
# Copy and paste the passkey from your Google account
# You should use the same account that is operating the Colab file
# Ignore if you aren't accessing this file in Google Colab

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Change working directory to data folder within parent directory

In [58]:
# Use os.walk to find the directory with the project folder and create a path object
# May take a while depending on number of sub-folders
# Ensure that the 'GxE with GEE' folder is in 'My Drive', not 'Shared with me'

parent_dir_name = 'GxE with GEE'

for dirpath, subdirs, files in os.walk('/content/drive/My Drive'):
  if parent_dir_name in subdirs:
    parent_path = dirpath + "/" + parent_dir_name

parent_path

'/content/drive/My Drive/Research/GxE with GEE'

In [59]:
# Set Data Path
## Change the destination to your Drive directory containing the folder 'raw_data.zip'
data_path_end = '/Colab Workspace/Data'
os.chdir(parent_path + data_path_end)

# Print the current working directory
!pwd

/content/drive/My Drive/Research/GxE with GEE/Colab Workspace/Data


## Step 1: Unzip raw_data.zip to raw_data folder



In [0]:
# Display files with `listdir` in os
os.listdir()

# Make sure your directory contains the raw_data.zip file

In [0]:
%%capture output
# Save contents of current directory to object
contents = os.listdir()

# If current directory does not contain folder 'raw_data', make directory
# then unzip contents of raw_data.zip to it
if 'raw_data' not in contents : 
  os.mkdir('raw_data')
  !unzip -o {'raw_data.zip'} 

## Step 2: Make a proc_data folder and copy files to it

In [0]:
# Save contents of current directory to object
contents = os.listdir()

# If current directory does not have folder 'proc_data', make directory
if 'proc_data' not in contents : 
  os.mkdir('proc_data')

# os.listdir()

In [0]:
# create a function to copy all files from one directory to another
def copyAllFilesinDir(srcDir, dstDir):
    # Check if both the are directories
    if os.path.isdir(srcDir) and os.path.isdir(dstDir) :
        # Iterate over all the files in source directory
        for filePath in glob.glob(srcDir + '/*'):
            # Move each file to destination Directory
            shutil.copy(filePath, dstDir);
    else:
        print("srcDir & dstDir should be Directories")

In [41]:
# Copy all files from raw_data to proc_data
# Note: It may take a minute for the files to appear in Drive

sourceDir = 'raw_data'
destDir = 'proc_data'
    
copyAllFilesinDir(sourceDir, destDir)

os.listdir('proc_data')[0:4]

['4ESWYT.zip', '6ESWYT.zip', '11HTWYT.zip', '2HTYWT.zip']

## Step 3: Unzip data folders into separate files based on file name

In [43]:
# Count number of files to unzip in proc_data
file_count = len(os.listdir('proc_data'))

# Create a list of directory paths in proc_data folder
files_in_proc_data = glob.glob('proc_data' + '/*')
files_in_proc_data[0:4]

['proc_data/4ESWYT.zip',
 'proc_data/6ESWYT.zip',
 'proc_data/11HTWYT.zip',
 'proc_data/2HTYWT.zip']

In [44]:
# Iterate over all files in proc_data to make folders with the file names

for idx,f in enumerate(files_in_proc_data):
  print(idx, " of ", file_count)
  basename = os.path.basename(f)
  dirname = os.path.splitext(basename)[0]
  if dirname not in os.listdir('proc_data') : 

    os.mkdir(os.path.join('proc_data', dirname))

# then unzip the files to the new folders
    !unzip -o -j {f} -d {os.path.join('proc_data', dirname)} -x "__MACOSX/*"
# before finally removing the original files
    os.remove(f)

  # Note: the ! passes unzip as a command prompt, not a Python function
  # https://www.lifewire.com/examples-linux-unzip-command-2201157
    # -o overwrites files of the same name
    # -j ignores subdirectory structures
    # -d specifies the destination directory
    # -x excludes files with the following folder structure (to ignore MACOSX files)

# May take a few minutes due to large number of zip files

0  of  108
Archive:  proc_data/4ESWYT.zip
  inflating: proc_data/4ESWYT/4TH ESWYT_EnvData.xls  
  inflating: proc_data/4ESWYT/4TH ESWYT.xls  
  inflating: proc_data/4ESWYT/4TH ESWYT_Genotypes_Data.xls  
  inflating: proc_data/4ESWYT/4TH ESWYT_RawData.xls  
  inflating: proc_data/4ESWYT/4TH ESWYT_MeanVal.xls  
  inflating: proc_data/4ESWYT/4TH ESWYT_GrnYld.xls  
  inflating: proc_data/4ESWYT/4TH ESWYT_Loc_data.xls  
1  of  108
Archive:  proc_data/6ESWYT.zip
  inflating: proc_data/6ESWYT/6TH ESWYT.xls  
  inflating: proc_data/6ESWYT/6TH ESWYT_EnvData.xls  
  inflating: proc_data/6ESWYT/6TH ESWYT_Genotypes_Data.xls  
  inflating: proc_data/6ESWYT/6TH ESWYT_GrnYld.xls  
  inflating: proc_data/6ESWYT/6TH ESWYT_Loc_data.xls  
  inflating: proc_data/6ESWYT/6TH ESWYT_MeanVal.xls  
  inflating: proc_data/6ESWYT/6TH ESWYT_RawData.xls  
caution: excluded filename not matched:  __MACOSX/*
2  of  108
Archive:  proc_data/11HTWYT.zip
  inflating: proc_data/11HTWYT/11HTWYT_Germplasm_DOIs.tab  
  infla

## Step 4: Make a Dataframe of Trials with path to Data folder

In [45]:
# Create a list of filenames for indexing
trial_list = []
for f in files_in_proc_data:
  basename = os.path.basename(f)
  dirname = os.path.splitext(basename)[0]
  trial_list.append(dirname)

trial_list[0:4]

['4ESWYT', '6ESWYT', '11HTWYT', '2HTYWT']

In [46]:
data_dir = pd.DataFrame(trial_list, columns = ['folder'])
data_dir['path'] = 'proc_data/' + data_dir['folder']
data_dir['trial_type'] = data_dir['folder'].str.findall('\D+').str.get(0)
data_dir['iteration'] = data_dir['folder'].str.findall('\d+').str.get(0)
data_dir.head()

,folder,path,trial_type,iteration
0,4ESWYT,proc_data/4ESWYT,ESWYT,4
1,6ESWYT,proc_data/6ESWYT,ESWYT,6
2,11HTWYT,proc_data/11HTWYT,HTWYT,11
3,2HTYWT,proc_data/2HTYWT,HTYWT,2
4,17HTWYT,proc_data/17HTWYT,HTWYT,17


In [47]:
data_dir['folder'].unique()

array(['4ESWYT', '6ESWYT', '11HTWYT', '2HTYWT', '17HTWYT', '10HTYWT',
       '16HTWYT', '1HTWYT', '12HTWYT', '3HTYWT', '15HTWYT', '14HTWYT',
       '4HTYWT', '5HTYWT', '2ESWYT', '23ESWYT', '13HTWYT', '3ESWYT',
       '8ESWYT', '9ESWYT', '5ESWYT', '7ESWYT', '22ESWYT', '6HTYWT',
       '7HTYWT', '9HRWYT', '3HRWYT', '8HTYWT', '10HRWYT', '19HRWYT',
       '6HRWYT', '26HRWYT', '18HRWYT', '11HRWYT', '8HRWYT', '1HRWYT',
       '20HRWYT', '25HRWYT', '24HRWYT', '21HRWYT', '17HRWYT', '2HRWYT',
       '16HRWYT', '4HRWYT', '23HRWYT', '15HRWYT', '13HRWYT', '14HRWYT',
       '22HRWYT', '7HRWYT', '39ESWYT', '12HRWYT', '37ESWYT', '32ESWYT',
       '35ESWYT', '38ESWYT', '28ESWYT', '36ESWYT', '33ESWYT', '26ESWYT',
       '24ESWYT', '30ESWYT', '29ESWYT', '34ESWYT', '31ESWYT', '25ESWYT',
       '21ESWYT', '20ESWYT', '18ESWYT', '17ESWYT', '19ESWYT', '13ESWYT',
       '16ESWYT', '1ESWYT', '15ESWYT', '14ESWYT', '11ESWYT', '2SAWYT',
       '10ESWYT', '12ESWYT', '10SAWYT', '7SAWYT', '4SAWYT', '3SAWYT',
       

## Step 5: Join Environmental, Location, and Genotype Data to Yield Files

### Example of Each Data Input Type

In [48]:
# Example of files: Location Data
test = data_dir['folder'][0]
contents1 = os.listdir('proc_data/' + test)
for file in contents1:
      if file.endswith('Loc_data.xlsx'):
        loc = pd.read_excel('proc_data/' + test + '/' + file, encoding='latin-1')
      if file.endswith('Loc_data.xls'):
        loc = pd.read_table('proc_data/' + test + '/' + file, encoding='latin-1')

loc["Lat_degress"]=loc['Lat_degress'].replace(regex=r"\..*",value="")
loc['lat_coord'] = (loc['Lat_degress'] + (loc['Lat_minutes'] / 100)).astype('float32')
loc.loc[loc['Latitud'] == 'S', 'lat_coord'] = loc['lat_coord'] * -1
loc.drop(['Lat_degress', 'Lat_minutes','Latitud'],axis=1,inplace=True)

loc["Long_degress"]=loc['Long_degress'].replace(regex=r"\..*",value="")
loc['long_coord'] = (loc['Long_degress'] + (loc['Long_minutes'] / 100)).astype('float32')
loc.loc[loc['Longitude'] == 'W', 'long_coord'] = loc['long_coord'] * -1
loc.drop(['Long_degress', 'Long_minutes','Longitude'],axis=1,inplace=True)

loc.head()

,Loc_no,Country,Loc. Description,Institute Name,Cooperator,Altitude,lat_coord,long_coord
0,11101,ZIMBABWE,HARARE,Crop Breeding Inst.,Bruce Mutari,1448,-17.410000,30.250000
1,19105,EGYPT,SHANDAWEEL,Agric. Res. Center,Abdel/A. Maboud/Shafi Ali,57,26.360001,31.400000
2,19207,LIBYA,KUFRA PRODUCTION PROJECT,Kufra Production,Moftah S. Elhata,415,25.000000,23.000000
3,19402,TUNISIA,BEJA,INRAT,Ali Maamouri,150,36.439999,9.080000
4,20203,CYPRUS,ATHALASSA A.R.I.,Min. of Agr.& Nat. R.,Ioannis Papadopoulos,142,35.080002,33.240002


In [49]:
# Example of files: Environmental data
contents1 = os.listdir('proc_data/' + test)
for file in contents1:
      if file.endswith('EnvData.xlsx'):
        env = pd.read_excel('proc_data/' + test + '/' + file, encoding='latin-1')
      if file.endswith('EnvData.xls'):
        env = pd.read_table('proc_data/' + test + '/' + file, encoding='latin-1')

env.head()

,Trial name,Occ,Loc_no,Country,Loc_desc,Cycle,Trait No,Trait name,Value,Unit
0,4ESWYT,1,19105,EGYPT,SHANDAWEEL,82-83,305,COOPERATOR_NOTE_TAKER,"MOUSA GERGIS, MOUSTAFA AZAB, FARGHAL HEFNAWY",TEXT
1,4ESWYT,2,19207,LIBYA,KUFRA PRODUCTION PROJECT,82-83,305,COOPERATOR_NOTE_TAKER,DIRECTOR RESEARCH K.P.P.,TEXT
2,4ESWYT,3,19402,TUNISIA,BEJA,82-83,305,COOPERATOR_NOTE_TAKER,"M. DEGHAN, A. R. MAAMOURI",TEXT
3,4ESWYT,4,11101,ZIMBABWE,HARARE,82-83,305,COOPERATOR_NOTE_TAKER,N. MASHIRINGWANI,TEXT
4,4ESWYT,5,22001,BANGLADESH,JOYDEBPUR,82-83,305,COOPERATOR_NOTE_TAKER,S. M. AHMED AND A.B.S. HOSSAIN,TEXT


In [50]:
traits = env['Trait name']
units = env['Unit']
agg_trait_dict = pd.DataFrame([traits, units]).T
agg_trait_dict.drop_duplicates('Trait name', inplace=True)

agg_trait_dict

,Trait name,Unit
0,COOPERATOR_NOTE_TAKER,TEXT
28,HARVEST_FINISHING_DATE,date
54,HARVEST_STARTING_DATE,date
56,IRRIGATED,Y/N
86,LENGTH_OF_ROWS_HARVESTED,m
114,LENGTH_OF_ROWS_SOWN,m
143,NO_OF_ROWS_HARVESTED,integer
171,NO_OF_ROWS_SOWN,integer
200,OTHER_MICRONUTRIENT_TOXICITY/DEFICIENCY_Y/N,Y/N
230,SOIL_ALUMINIUM_TOXICITY,Y/N


In [51]:
# Check if each location + Occ combination contains all traits
# Many locations seem to be commonly missing at least a couple of traits

env['Loc_ID'] = env['Loc_no'].astype(str) + '_' + env['Occ'].astype(str)
env.groupby('Loc_ID')['Trait name'].nunique()

Loc_ID
11101_4     12
19105_1     12
19207_2     12
19402_3     12
20203_15    12
21103_17    12
21207_18     8
21601_16    12
22001_5     12
22231_6     12
22607_9     12
22611_8     12
24029_7     12
40102_19    12
41103_27     5
42104_28    12
42203_29    12
45104_10    12
50204_30    13
51004_20    11
51204_23    12
51403_25    11
51501_26    12
53002_22    11
53005_21    11
53101_24    13
53201_14    12
61505_12    12
65106_11    12
65301_13    12
Name: Trait name, dtype: int64

In [52]:
# Example of files: Grain yield data

contents1 = os.listdir('proc_data/' + test)
for file in contents1: 
      if file.endswith('GrnYld.xlsx'):
        grn_yld = pd.read_excel('proc_data/' + test + '/' + file,  encoding='latin-1')
      if file.endswith('GrnYld.xls'):
        grn_yld = pd.read_table('proc_data/' + test + '/' + file,  encoding='latin-1')

grn_yld.rename(columns = {'Value':'Grain_Yield'}, inplace=True)
grn_yld.drop(['Trait_no', 'Trait name'], axis=1, inplace=True)

grn_yld['Unique_ID'] = test + "_" + grn_yld['Loc_no'].astype(str) + "_" + grn_yld['Cid'].astype(str) + "_" + grn_yld['Sid'].astype(str)

grn_yld.set_index('Unique_ID',inplace=True)

grn_yld

,Trial name,Occ,Loc_no,Country,Loc_desc,Cycle,Cid,Sid,Gen_name,Grain_Yield,EMS,SE,Unit
Unique_ID,,,,,,,,,,,,,
4ESWYT_19105_254741_5,4ESWYT,1,19105,EGYPT,SHANDAWEEL,82-83,254741,5,R37/GHL121//KAL/BB,6.25,0,0,t/ha
4ESWYT_19105_9068_2264,4ESWYT,1,19105,EGYPT,SHANDAWEEL,82-83,9068,2264,VORONA,4.071,0,0,t/ha
4ESWYT_19105_7673_65,4ESWYT,1,19105,EGYPT,SHANDAWEEL,82-83,7673,65,TANAGER,5.298,0,0,t/ha
4ESWYT_19105_7691_43,4ESWYT,1,19105,EGYPT,SHANDAWEEL,82-83,7691,43,VEERY #8,8.179,0,0,t/ha
4ESWYT_19105_7691_18,4ESWYT,1,19105,EGYPT,SHANDAWEEL,82-83,7691,18,GENARO T 81,7.917,0,0,t/ha
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4ESWYT_50204_7799_11,4ESWYT,30,50204,BRAZIL,LONDRINA,82-83,7799,11,LIRA,1.911,0,0,t/ha
4ESWYT_50204_8122_4,4ESWYT,30,50204,BRAZIL,LONDRINA,82-83,8122,4,BUC/FLK,1.556,0,0,t/ha
4ESWYT_50204_6225_61,4ESWYT,30,50204,BRAZIL,LONDRINA,82-83,6225,61,CANANEA 79,0.878,0,0,t/ha


### Loop Over All Files and Aggregate Data

In [53]:
# Index into each folder within proc_data
# Join location data and environmental data to the grain yield data on 'Loc_no' column

# Set up a progress bar for the loop
pbar = ProgressBar()

for f in pbar(data_dir['path']) :
  # Step 1: Read yield, location, environment, and genotype files into pandas dataframes
  contents = os.listdir(f)
  try:
    for file in contents:
      if file.endswith('GrnYld.xlsx'):
        grn_yld = pd.read_excel(f + '/' + file,  encoding='latin-1')
      if file.endswith('GrnYld.xls'):
        grn_yld = pd.read_table(f + '/' + file,  encoding='latin-1')
      if file.endswith('Loc_data.xlsx'):
        loc = pd.read_excel(f + '/' + file, encoding='latin-1')
      if file.endswith('Loc_data.xls'):
        loc = pd.read_table(f + '/' + file, encoding='latin-1')
      if file.endswith('EnvData.xlsx'):
        env = pd.read_excel(f + '/' + file, encoding='latin-1')
      if file.endswith('EnvData.xls'):
        env = pd.read_table(f + '/' + file, encoding='latin-1')
      if file.endswith('Genotypes_Data.xlsx'):
        geno = pd.read_excel(f + '/' + file, encoding='latin-1')
      if file.endswith('Genotypes_Data.xls'):
        geno = pd.read_table(f + '/' + file, encoding='latin-1') 

  # Step 2: Create Dictionary of Environmental Traits and Units
    traits = env['Trait name']
    units = env['Unit']
    trait_dict = pd.DataFrame([traits, units]).T
    trait_dict.drop_duplicates('Trait name', inplace=True)

    agg_trait_dict = pd.concat([agg_trait_dict, trait_dict], axis=0, copy=False)
    agg_trait_dict = agg_trait_dict.drop_duplicates('Trait name')

  # Step 3a: Create dataframe of location specific columns
    env['Loc_ID'] = env['Loc_no'].astype(str) + '_' + env['Occ'].astype(str)
    env_ind = env.sort_values('Loc_ID')
    env_ind.drop(['Trait No', 'Trait name', 'Value', 'Unit'], axis = 1, inplace=True)
    env_ind.drop_duplicates('Loc_ID', inplace=True)
    env_ind = env_ind.set_index('Loc_ID')

  # Step 3b: Reshape environment data to wide and join pivoted traits to location specific data
    env['Pivot'] = env['Loc_ID'].astype(str) + "_" + env['Trait name'].astype(str)
    env.drop_duplicates('Pivot', inplace=True)
    env_piv = env.pivot(index='Loc_ID', columns='Trait name', values='Value')
    env_con = env_ind.join(env_piv, on='Loc_ID')

  # Step 4: Create latitude and longitude coordinates in loc file
    loc["Lat_degress"]=loc['Lat_degress'].replace(regex=r"\..*",value="")
    loc['lat_coord'] = (loc['Lat_degress'] + (loc['Lat_minutes'] / 100)).astype('float32')
    loc.loc[loc['Latitud'] == 'S', 'lat_coord'] = loc['lat_coord'] * -1
    loc.drop(['Lat_degress', 'Lat_minutes','Latitud'],axis=1,inplace=True)

    loc["Long_degress"]=loc['Long_degress'].replace(regex=r"\..*",value="")
    loc['long_coord'] = (loc['Long_degress'] + (loc['Long_minutes'] / 100)).astype('float32')
    loc.loc[loc['Longitude'] == 'W', 'long_coord'] = loc['long_coord'] * -1
    loc.drop(['Long_degress', 'Long_minutes','Longitude'],axis=1,inplace=True)

  # Step 5: Merge environment (many) concatenated dataframe to location (1) dataframe
    loc.drop_duplicates('Loc_no', inplace=True)
    env_loc = pd.merge(left = env_con, right = loc, on="Loc_no", how='inner', validate='m:1')

  # Step 6a: Replace grain yield columns with single column
    grn_yld.rename(columns = {'Value':'Grain_Yield'}, inplace=True)
    grn_yld.drop(['Trait_no', 'Trait name'], axis=1, inplace=True)

  # Step 6b: Merge all location and environmental data (1) to grain yield (many)
    env_loc.drop_duplicates('Loc_no', inplace=True)
    yld_env_loc = pd.merge(left = grn_yld, right = env_loc, on="Loc_no", validate= 'm:1')

  # Step 7: Merge genotype data (1) to grain yield (many)
    yld_env_loc['Geno_ID'] = yld_env_loc['Cid'].astype(str) + '_' + yld_env_loc['Sid'].astype(str)
    geno['Geno_ID'] = geno['Cid'].astype(str) + '_' + geno['Sid'].astype(str)

    geno.drop_duplicates('Geno_ID', inplace=True)
    yld_env_loc_geno = pd.merge(left = yld_env_loc, right = geno, on='Geno_ID', validate='m:1')
    yld_env_loc_geno['Unique_ID'] = f[10:] + "_" + yld_env_loc_geno['Loc_no'].astype(str) + "_" + yld_env_loc_geno['Geno_ID'].astype(str)

    yld_env_loc_geno.set_index('Unique_ID',inplace=True, drop=False)

  # Step 8: Write pickle of merged data to folders
    yld_env_loc_geno.to_pickle(f + '/' + f[10:] + '_merged.pkl')

  except:
    # inform the user of the error
    print("Error encountered while parsing chunk {}".format(f))


100% (108 of 108) |######################| Elapsed Time: 0:00:37 Time:  0:00:37


In [54]:
# Complete dictionary of trait names and units from all trials (126 at last run)
agg_trait_dict.to_pickle('agg_trait_dict.pkl')
agg_trait_dict.to_csv('agg_trait_dict.csv')
agg_trait_dict

,Trait name,Unit
0,COOPERATOR_NOTE_TAKER,TEXT
28,HARVEST_FINISHING_DATE,date
54,HARVEST_STARTING_DATE,date
56,IRRIGATED,Y/N
86,LENGTH_OF_ROWS_HARVESTED,m
...,...,...
311,K_FERTILIZER_APPLIED_OLD,mark
330,LOCAL_CHECK,TEXT
435,N_FERTILIZER_APPLIED_OLD,mark
508,P_FERTILIZER_APPLIED_OLD,mark


## Step 6: Aggregate Data into One File

In [60]:
# First make a dataframe with the first trial merged pkl
aggregated_data = pd.read_pickle(data_dir['path'][0] + '/' + data_dir['path'][0][10:] + '_merged.pkl')

# Set up a progress bar for the loop
pbar = ProgressBar()

# Then use a loop to append each subsequent merged pkl to the aggregated dataframe
for f in pbar(data_dir['path'][1:]) :
  merged_data = pd.read_pickle(f + '/' + f[10:] + '_merged.pkl')
  aggregated_data = pd.concat([aggregated_data, merged_data], axis=0, sort=False)
  aggregated_data.set_index('Unique_ID')

100% (107 of 107) |######################| Elapsed Time: 0:01:41 Time:  0:01:41


In [61]:
#Terry's check
check = aggregated_data
check.columns=aggregated_data.columns.str.upper()
check["HARVEST_FINISHING_DATE"] = pd.to_datetime(aggregated_data["HARVEST_FINISHING_DATE"], errors="coerce")
check.loc[aggregated_data["HARVEST_FINISHING_DATE"]== aggregated_data['HARVEST_FINISHING_DATE'].max()]
#pd.to_datetime(aggregated_data[aggregated_data["LOC_NO"].iloc[:,0] != 42148]["HARVEST_FINISHING_DATE"]).max()

,TRIAL NAME_X,OCC_X,LOC_NO,COUNTRY,LOC_DESC_X,CYCLE_X,CID_X,SID_X,GEN_NAME,GRAIN_YIELD,EMS,SE,UNIT,TRIAL NAME_Y,OCC_Y,COUNTRY_X,LOC_DESC_Y,CYCLE_Y,COOPERATOR_NOTE_TAKER,HARVEST_FINISHING_DATE,HARVEST_STARTING_DATE,IRRIGATED,LENGTH_OF_ROWS_HARVESTED,LENGTH_OF_ROWS_SOWN,NO_OF_ROWS_HARVESTED,NO_OF_ROWS_SOWN,OTHER_MICRONUTRIENT_TOXICITY/DEFICIENCY_Y/N,SOIL_ALUMINIUM_TOXICITY,SOWING_DATE,SPACE_BTN_ROWS_SOWN,YIELD_FACTOR,COUNTRY_Y,LOC. DESCRIPTION,INSTITUTE NAME,COOPERATOR,ALTITUDE,LAT_COORD,LONG_COORD,GENO_ID,CID_Y,...,UNNAMED: 7,UNNAMED: 8,UNNAMED: 9,UNNAMED: 10,UNNAMED: 11,UNNAMED: 12,UNNAMED: 13,UNNAMED: 14,UNNAMED: 15,UNNAMED: 16,UNNAMED: 17_Y,UNNAMED: 18_Y,UNNAMED: 19_Y,UNNAMED: 20_Y,UNNAMED: 21_Y,UNNAMED: 22_Y,UNNAMED: 23_Y,SOWING_DATE_TEXT,HARVEST_FINISHING_DATE_TEXT,FERTILIZER_TEXT_123,HARVEST_STARTING_DATE_TEXT,CALCULATED_OF_TOTAL_WATER_APPLIED_BY_IRRIGATION,EMERGENCE_DATE_TEXT,FERTILIZER_DATE_TEXT_3,UNNAMED: 17,UNNAMED: 18,UNNAMED: 19,UNNAMED: 20,UNNAMED: 21,UNNAMED: 22,UNNAMED: 23,DISEASE_DEVELOPMENT_IN_GENERAL,ESTIMATE_OF_TOTAL_WATER_APPLIED_BY_IRRIGATION,K_FERTILIZER_APPLIED_OLD,LOCAL_CHECK,N_FERTILIZER_APPLIED_OLD,PRECIPITATION_AVAILABLE_TO_CROP_AFTER_SOWING_OLD,P_FERTILIZER_APPLIED_OLD,ENTRY_X,ENTRY_Y
Unique_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
17HTWYT_42148_61665_1,17TH HIGH TEMPERATURE WHEAT YT,19,42148,MEXICO,CENEB - LATE HEAT,2018,61665,1,LOCAL CHECK,5.468,0.162,0.403,t/ha,17TH HIGH TEMPERATURE WHEAT YT,19,MEXICO,CENEB - LATE HEAT,2018,RAVI P. SINGH,2049-06-12,Jun 12 2019,YES,NaN,NaN,NaN,NaN,NO,NO,Feb 3 2019,NaN,2.2321,MEXICO,CENEB - LATE HEAT,CIMMYT,Ravi Singh,38.0,27.26,-109.57,61665_1,61665.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17HTWYT_42148_520792_37,17TH HIGH TEMPERATURE WHEAT YT,19,42148,MEXICO,CENEB - LATE HEAT,2018,520792,37,NADI #2,5.721,0.162,0.403,t/ha,17TH HIGH TEMPERATURE WHEAT YT,19,MEXICO,CENEB - LATE HEAT,2018,RAVI P. SINGH,2049-06-12,Jun 12 2019,YES,NaN,NaN,NaN,NaN,NO,NO,Feb 3 2019,NaN,2.2321,MEXICO,CENEB - LATE HEAT,CIMMYT,Ravi Singh,38.0,27.26,-109.57,520792_37,520792.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17HTWYT_42148_536182_80,17TH HIGH TEMPERATURE WHEAT YT,19,42148,MEXICO,CENEB - LATE HEAT,2018,536182,80,QUAIU #1/SUP152,6.273,0.162,0.403,t/ha,17TH HIGH TEMPERATURE WHEAT YT,19,MEXICO,CENEB - LATE HEAT,2018,RAVI P. SINGH,2049-06-12,Jun 12 2019,YES,NaN,NaN,NaN,NaN,NO,NO,Feb 3 2019,NaN,2.2321,MEXICO,CENEB - LATE HEAT,CIMMYT,Ravi Singh,38.0,27.26,-109.57,536182_80,536182.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17HTWYT_42148_562658_25,17TH HIGH TEMPERATURE WHEAT YT,19,42148,MEXICO,CENEB - LATE HEAT,2018,562658,25,WBLL1*2/SHAMA//KACHU/3/PRL/6/SAUAL/4/CROC_1/AE...,5.236,0.162,0.403,t/ha,17TH HIGH TEMPERATURE WHEAT YT,19,MEXICO,CENEB - LATE HEAT,2018,RAVI P. SINGH,2049-06-12,Jun 12 2019,YES,NaN,NaN,NaN,NaN,NO,NO,Feb 3 2019,NaN,2.2321,MEXICO,CENEB - LATE HEAT,CIMMYT,Ravi Singh,38.0,27.26,-109.57,562658_25,562658.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17HTWYT_42148_571824_53,17TH HIGH TEMPERATURE WHEAT YT,19,42148,MEXICO,CENEB - LATE HEAT,2018,571824,53,MUTUS*2/KINGBIRD #1/3/KSW/SAUAL//SAUAL,5.76,0.162,0.403,t/ha,17TH HIGH TEMPERATURE WHEAT YT,19,MEXICO,CENEB - LATE HEAT,2018,RAVI P. SINGH,2049-06-12,Jun 12 2019,YES,NaN,NaN,NaN,NaN,NO,NO,Feb 3 2019,NaN,2.2321,MEXICO,CENEB - LATE HEAT,CIMMYT,Ravi Singh,38.0,27.26,-109.57,571824_53,571824.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [0]:
# Check on Index issues
# Check if concatenation preserves all columns

# Merges seem to have added _x and _y to columns, might be causing them not to append

# Issue: Many trials have different environmental variable names, which may not append properly
# Need to investigate all the NaN values and Unnamed columns

# Grain yields seem to have appended properly


In [0]:
# Convert column names to uppercase, drop harvest date outlier
aggregated_data.columns=aggregated_data.columns.str.upper()
outlier = aggregated_data['HARVEST_FINISHING_DATE'].max()
toDrop = aggregated_data.loc[aggregated_data['HARVEST_FINISHING_DATE'] == outlier]
aggregated_data = aggregated_data.drop(toDrop.index)

In [0]:
aggregated_data.to_pickle('aggregated_data.pkl')

In [0]:
aggregated_data.to_csv('aggregated_data.csv')

In [0]:
HTWYT_merged_data = pd.read_pickle('proc_data/17HTWYT/17HTWYT_merged.pkl')

In [0]:
HTWYT_merged_data['HARVEST_FINISHING_DATE'].unique()

array(['2019-04-26T00:00:00.000000000', '2019-04-20T00:00:00.000000000',
       '2019-04-15T00:00:00.000000000', '2019-05-27T00:00:00.000000000',
       '2019-06-24T00:00:00.000000000', '2019-06-20T00:00:00.000000000',
       '2019-05-09T00:00:00.000000000', '2019-04-22T00:00:00.000000000',
       '2019-04-13T00:00:00.000000000', '2049-06-12T00:00:00.000000000',
       '2019-04-30T00:00:00.000000000',                           'NaT',
       '2019-04-27T00:00:00.000000000', '2019-04-23T00:00:00.000000000',
       '2019-05-05T00:00:00.000000000', '2019-04-01T00:00:00.000000000',
       '2019-05-16T00:00:00.000000000', '2019-05-10T00:00:00.000000000',
       '2019-03-14T00:00:00.000000000', '2019-05-18T00:00:00.000000000',
       '2019-04-04T00:00:00.000000000', '2019-05-23T00:00:00.000000000',
       '2019-05-22T00:00:00.000000000', '2019-03-19T00:00:00.000000000',
       '2019-04-21T00:00:00.000000000', '2019-04-24T00:00:00.000000000',
       '2019-06-05T00:00:00.000000000', '2019-06-11